In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from os.path import basename
import requests
from webdriver_manager.chrome import ChromeDriverManager
driver  = webdriver.Chrome(ChromeDriverManager().install())





In [ ]:
!pip install selenium
!pip install pandas
!pip install bs4
!pip install webdriver_manager


In [63]:
#entramos a amazon.es
driver.get('http:amazon.es')

In [ ]:
#aceptamos las cookies
driver.find_element_by_name('accept').click()

In [76]:
#escribimos en el buscador la palabra 'Iphone'
driver.find_elements_by_id("twotabsearchtextbox")[0].send_keys("Iphone")

C:\Users\USUARIO\AppData\Local\Temp/ipykernel_14672/3442902271.py:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_id("twotabsearchtextbox")[0].send_keys("Iphone")


In [65]:
#enviamos la palabra 'le damos a buscar'
driver.find_elements_by_id("twotabsearchtextbox")[0].submit()

C:\Users\USUARIO\AppData\Local\Temp/ipykernel_14672/3093474211.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_id("twotabsearchtextbox")[0].submit()


In [66]:
#varible que usaremos para buscar diferentes productos,
#cambiando la variable buscamos otro producto
keyword="Iphone"

In [67]:
#buscamos a traves de la url, pasando como parametro la variable
#previamente definida 
driver.get("https://www.amazon.es/s?k="+keyword)

In [68]:
#creamos una variable de conexión con la url de la página
content = driver.page_source

In [69]:
#descargamos la pagina 
soup= BeautifulSoup(content)

In [70]:
#buscamos un elemento en la pagina con la classe 'a-section a-spacing-medium'
resultado = soup.findAll(class_ = "a-section a-spacing-medium")

In [71]:
#creamos un data frame con los elementos titulo,precio e imagen, 
#que luego llenaremos con los datos de la pagina
df_lista_productos = pd.DataFrame(columns=["titulo","precio","imagen"])

In [72]:
#le decimos que usando la ruta actual de donde se ejecuta
#este fichero, cre un directorio llamado Fotos + la palabra
#clave de busqueda, si no existe ya.

if not os.path.exists("Fotos "+keyword):
    os.makedirs("Fotos "+keyword)

In [73]:
#asignamos la variable carpeta el nombre del directorio,donde
#se guardarán las fotos
carpeta = "Fotos "+keyword+"/"

In [74]:
df_lista_productos = pd.DataFrame(columns=["titulo","precio","imagen"])

#creamos un boocle que recore la variable resultado y saca los elementos
# nombre de producto,precio y imagen, luego los introduce en el data frame
for i in resultado:
    titulo_producto = i.find(class_ = "a-size-base-plus a-color-base a-text-normal").get_text()
    precio_producto = i.find(class_ = "a-price-whole")
    imagen_producto = i.find("img")
    if precio_producto != None:
        precio_producto=precio_producto.get_text()
    else:
        precio_producto="No existe precio"
    
    fichero = open(carpeta+basename(imagen_producto.attrs["src"]),"wb")
    fichero.write(requests.get(imagen_producto.attrs["src"]).content)

    df_lista_productos = df_lista_productos.append({"titulo":titulo_producto,
                                                     "precio": precio_producto,
                                                     "imagen": "<img src='"+imagen_producto.attrs["src"]+"'>"},ignore_index=True)

In [75]:
#creamos un archivo csv a partir del dataframe del iphone
#tambien comvertimos el df en una tabla que podemos
#como html

df_lista_productos.to_csv(keyword+".csv")
df_lista_productos.to_html(keyword+".html", escape=False)